**Student first name:** *Hao* `  `  
**Student last name:** *Zhang*  
**Student ANU UNI ID:** *u5319850*

COMP4650 Social Media Assignment Submission (10 points)
===

1. Assignment questions related to tutorial 1: Constructing and analyzing a social network
---

This initial code is necessary for loading into this notebook's Python kernel the object necessary for this part of the assignment. Run the next cell before starting to work on the assignment questions below.

In [6]:
# necessary imports
import csv
import urllib2
import networkx as nx

# opening a local CSV file
to_read = file("./dataset.csv")  #use this line for a locally downloaded file
# or reading it directly from the specified URL
# url = 'http://rizoiu.eu/sna-lab-ipython/dataset.csv'
# to_read = urllib2.urlopen(url)
reader = csv.reader(to_read)

#  construct the networkx graph
G = nx.Graph()
for line in reader:
    if line[0] not in G:  G.add_node(line[0])
    if line[1] not in G:  G.add_node(line[1])
    G.add_edge(line[0], line[1])

**(1 point) Assignment question #1.1:** Find another way of determining the number of users, by applying a (networkx) graph method.

In [5]:
print "The social graph contains %s users." % G.number_of_nodes()

**(0.5 points) Assignment question #1.2:** What is the minimum number of introductions required for the user `'137056623'` to reach any other user? 

In [6]:
no_hops = max([len(nx.shortest_path(G, source = '137056623', target = x)) - 1 for x in G.nodes()])
print "Using %d introductions, '137056623' can reach anyone in the network" % no_hops

**(0.5 points) Assignment question #1.3:** What is the minimum number of introductions required for the any user to reach any other user in the  network?  

**HINT:** study the `shortest_path_length` method description.

In [7]:
no_hops = max (max([len(nx.shortest_path(G, source = y, target = x)) - 1 for x in G.nodes()]) for y in G.nodes())
print "Using %d introductions, any user can reach any other user in the network" % no_hops

**(0.5 points) Assignment question #1.4:** The _diameter_ $d$ of a graph is the maximum eccentricity of any node, $d = \max_{v} \epsilon(v)$. Give two ways to compute the diameter of the social network G (one using the calculated eccentricity values calculated earlier and another one using the dedicated `networkx` function).

In [7]:
import operator

ec = nx.eccentricity(G)

print max(ec.values())
## OR 
print nx.diameter(G)

18
18


**(0.5 points) Assignment question #1.5:** Determine ALL most central node(s) and ALL most central edge(s), with respect to the node and edge betweenes centrality. Remember that two or more nodes/edges may have the same centrality score and they ALL need to be determined for this assignment.  
**Hint:** `networkx` already contains dedicated function to compute node and edge betweenness centrality scores.

In [16]:
print "The most betweenness central users:",
bc = nx.betweenness_centrality(G)
# the greatest centrality value
max_node_centrality = max(bc.iteritems(), key=operator.itemgetter(1))[1]

kv_pairs = [(a,b) for (a,b) in bc.iteritems()]
for kv in kv_pairs:
    if kv[1] == max_node_centrality:
        print "'"+str(kv[0])+"'",
print

print "The most betweenness central edges:",
ebc = nx.edge_betweenness_centrality(G)
# the greatest centrality value
max_edge_centrality = max(ebc.iteritems(), key = operator.itemgetter(1))[1]

kv_pairs = [(a,b) for (a,b) in ebc.iteritems()]
for kv in kv_pairs:
    if kv[1] == max_edge_centrality:
        print "'"+str(kv[0])+"'",
print

The most betweenness central users: '34507480'
The most betweenness central edges: '(369751802, 34507480)'


In [ ]:
The most betweenness central users: '34507480'
The most betweenness central edges: '(369751802, 34507480)'

2. Assignment questions related to the tutorial 2: constructing a network from real data
---

**(1 point) Assignment question #2.1:** Starting from [the same bzipped JSON twitter dataset](http://rizoiu.eu/sna-lab-ipython/twitter-dump.json.bz2), construct the social graph based on the reply relation. Analyse (as shown in *Step 3*) which fields you require and give the Python code necessary for constructing the network.  
**NOTE:** all the subsequent questions of this subsection are to be solved on the graph constructed at this question.

We will construct the reply social graph as a directed graph. Nodes will be the users in our dataset (denoted by their user id). A directional edge will be placed between the users u1u1 and u2u2 (u1⟶u2u1⟶u2) if and only if u1u1 replies u2u2. More precisely, if a tweet exists in the dataset such that all of the following conditions are simultaneously fulfilled:
* the **user.id** (the user emitting the tweet) is u1u1;
* the **in_reply_to_user_id** (the user emitting the replied tweet) is u2u2.

In [9]:
import bz2, json
import networkx as nx

reader = bz2.BZ2File("./twitter-dump.json.bz2", mode="r")

# code for question.
DG = nx.DiGraph()

# code for constructing the Directed Graph `DG`
for line in reader:
    # load the JSON object from the read line
    jobj = json.loads(line)
    
    # if the current tweet is a reply
    if jobj[u'in_reply_to_user_id']:
        # determine user id
        user_id = jobj[u'user'][u'id']
        if isinstance(user_id, dict):
            user_id = user_id.values()[0]
        user_id = int(user_id)
        
        #determine the replied user id
        reply_user_id = jobj[u'in_reply_to_user_id']
        if isinstance(reply_user_id, dict):
            reply_user_id = reply_user_id.values()[0]
        reply_user_id = int(reply_user_id)
        
        # add the two nodes in the graph, if not already there
        if user_id not in DG:  DG.add_node(user_id)
        if reply_user_id not in DG:  DG.add_node(reply_user_id)
        DG.add_edge(user_id, reply_user_id)

**(1 point) Assignment question #2.2:** How many nodes and edges do you have in the resulted reply graph?

In [10]:
print "Number of nodes: %d" % DG.number_of_nodes()
print "Number of edges: %d" % DG.number_of_edges()

Number of nodes: 735
Number of edges: 622


**(1 point) Assignment question #2.3:** Using the `networkx` functions, filter the social graph to its *giant connected component*. 
Calculate for this new graph: i) the number of nodes, ii) the number of edges, iii) the radius and iv) the diameter.  
**HINT:** You can either construct a new graph which contains only the nodes and edges in the *giant connected component* or you can remove from your initial graph all the nodes and edges belonging to the other connected components.  
**NOTE:** This question should be solved on the UNDIRECTED version of the social graph. Considering that *DG* is the constructed directed networkx graph, do:

In [12]:
# create an undirected copy of our graph
G = DG.to_undirected()

# get the list of connected components, sorted by size
cc_List=sorted(nx.connected_component_subgraphs(G), key = len, reverse=True)

# we want the first component, the biggest
gcc = cc_List[0]

print "Number of nodes in giant connect component: %d" % gcc.number_of_nodes()
print "Number of edges in giant connect component: %d" % gcc.number_of_edges()
print "Radius of giant connect component: %d" % nx.radius(gcc)
print "Diameter of giant connect component: %d" % nx.diameter(gcc)

Number of nodes in giant connect component: 465
Number of edges in giant connect component: 467
Radius of giant connect component: 3
Diameter of giant connect component: 6


In [ ]:
Number of nodes in giant connect component: 465
Number of edges in giant connect component: 467
Radius of giant connect component: 3
Diameter of giant connect component: 6

**(1 point) Assignment question #2.4:** This assignment question is to calculate iterativelly the basic form of the PageRank on the reply social graph constructed previously. Considering that *DG* is the constructed directed networkx graph constructed at the **Assignment questions #2.1**. We will construct the *PageRank* on the largest [*weakly connected component*](http://en.wikipedia.org/wiki/Connectivity_%28graph_theory%29). This is constructed as shown hereafter.
Your job is to compute the PageRank, using the algorithm provided in the tutorial and exemplified in the lecture notes.
Use a maximum of $maxiter = 20$ iterations.

In [15]:
import operator
from itertools import islice

# get the list of weakly connected components, sorted by size
omp_List = sorted(nx.weakly_connected_component_subgraphs(DG), key = len, reverse=True)

# we want the first component, the biggest
WCDG = omp_List[0]

# get the number of nodes of the graph
n = len(WCDG.nodes())

# initialize the PR at moment 0 using a dictionary of pairs {node : score}
PR = {}
for node in WCDG.nodes():
    PR[node] = 1.0/n

# initialize variables
no_iter=0     # current iteration
iter_max=20   # maximum number of iterations
d = 0.7       # decay factor

while no_iter < iter_max: # iterate until maximum iterations
    # at each iteration, we compute a new version of the PageRank for each node
    new_PR = {}
    no_iter = no_iter + 1

    # calculate the PageRank of each node (`new_PR[node]`), based on the previous values (`PR[other_nodes]`)
    for node in WCDG.nodes():
        ## code for calculating new_PR[node].
        new_PR[node] = (1.0 - d) / n
        B = WCDG.predecessors(node)
        for s in B:
            N = len(WCDG.successors(s))
            new_PR[node] = new_PR[node] + 1.0 * d * PR[s] / N

    # at the end of iteration, replace old values of PR with the new ones
    PR = new_PR

# even if a dictionary is not sorted (it is just a collection) we sort it so that the next
# printing at the screen shows some interesting values
sorted_PR = sorted(PR.iteritems(), key=operator.itemgetter(1), reverse=True)

# and print it out the first 10 elements
list(islice(sorted_PR, 10))

[(34507480, 0.20229379652869947),
 (34359954, 0.004483870967741935),
 (153694176, 0.0026774193548387095),
 (336555582, 0.0010967741935483872),
 (271239188, 0.0009925558321466847),
 (1188471348, 0.0009925558321466847),
 (2502937470, 0.0009925558321466847),
 (261289587, 0.000870967741935484),
 (1456140289, 0.0006451612903225808),
 (217733121, 0.0006451612903225808)]

3. Assignment questions related to the tutorial 3: sentiment analysis
---

The following code is necessary to define the sentiment scoring function that was constructed in the tutorial. Run this before continuing with the assignments:

In [20]:
from nltk.stem.wordnet import WordNetLemmatizer
import nltk

# define the lemmatizer
lmtzr = WordNetLemmatizer()

# read the positive and negative lexicon in lists of words
positive_words = [lmtzr.lemmatize(line.strip().decode('utf-8')) for line in open('sentiment-lexicon-positive-words.txt')]
negative_words = [lmtzr.lemmatize(line.strip().decode('utf-8')) for line in open('sentiment-lexicon-negative-words.txt')]

# define the function that computes the sentiment score
def get_sentiment_score(text):
    # tokenize and lemmatize the current tweet
    tokens = nltk.word_tokenize(text)
    tweet = [lmtzr.lemmatize(x.lower()) for x in tokens if len(x) >= 3]

    # calculate the sentiment score
    score = 0
    for word in tweet:
        if word in positive_words:
            score = score + 1
        if word in negative_words:
            score = score - 1
    return score

**(1 point) Assignment question #3.1:** Use the `get_sentiment_score(text)`, the sentiment scoring function defined before, and calculate the sentiment polarity of the tweets in the Twitter JSON dataset used in [tutorial 2](http://nbviewer.ipython.org/url/rizoiu.eu/sna-lab-ipython/tutorial-2-construct-network-real-twitter-dump.ipynb). 
The dataset is [available to download here](http://rizoiu.eu/sna-lab-ipython/twitter-dump.json.bz2). 
Print the text of the 10 most positive and the 10 most negative tweets. 
We consider that a tweet $t_1$ is more positive than another tweet $t_2$ when score of the former is higher than the score of the latter ($score(t_1) > score(t_2)$). 
Similarly, a tweet $t_1$ is more negative than $t_2$ when $score(t_1) < score(t_2)$.  
**HINT:** Load the tweets one by one as seen in tutorial 2 and extract the text, which is found in the field *text* of each tweet.

In [24]:
import bz2, json

# open the bzipped dataset
reader = bz2.BZ2File("./twitter-dump.json.bz2", mode="r")

tweetList = []
# go line by line (ergo, tweet by tweet)
for line in reader:
    # load the JSON object from the read line
    jobj = json.loads(line)
    # get the text of the tweet
    text = jobj[u'text']
    # calculate sentiment score of the tweet
    score = get_sentiment_score(text)
    # below attributes are used for future questions
    userId = jobj[u'user'][u'id']
    # append to list as a tuple
    tweetList.append((text, userId, score))
# sort the tweets from lower score to higher
tweetList.sort(key=lambda x:x[2], reverse=True)

print "--- The 10 most positive tweets ---"
for i in xrange(10):
    print str(i+1)+". "+tweetList[i][0]
print

print "--- The 10 most nagative tweets ---"
for i in xrange(10):
    print str(i+1)+". "+tweetList[-i-1][0]

--- The 10 most positive tweets ---
1. @ArianaGrande Problem slays my life. Problem slays your life. Problem over everything. Problem slays. https://t.co/b0jyBCGpXn
2. TWEET THE FUCKING LINK WITH THE HASHTAG. NO ONES GONNA FUCKING GOOGLE THAT SHIT TO FIND IT. http://t.co/Ktrjp69N7E #WatchProblemOnVEVO
3. RT @_ilysm_Ari: ARIANATORS RT AND SPREAD THE WORLD
Problem broke an iTunes record time to break a vevo record. #WatchProblemOnVEVO https://…
4. I hate those comments that are negative in Ariana's #problemvideo FUCK OFF!! http://t.co/cl6hTuPRLr
5. Ariana Grande - Problem ft. Iggy Azalea: http://t.co/RJioyyg2ZK  Attention! lets try to get problem to break the record #WatchProblemOnVEVO
6. Ariana Grande - Problem ft. Iggy Azalea omg I died when I saw it amazingg!! #problem  http://t.co/4RXDEqqb7w
7. RT @zaynmgirl: one less problem without ya I got one less problem without ya I got one less problem without ya https://t.co/0tLoIbAVcc
8. One less problem without ya!
Ariana Grande - Problem f

**(1 point) Assignment question #3.2:** Based on the scores calculated in **Assignment #3.1**, determine the 3 most positive users. A user $u_1$ is more positive than a user $u_2$ if the dataset contains more positive tweets emitted by $u_1$ than tweets emitted by $u_2$. Formally:
$$ positivity(u_1) > positivity(u_2) \iff \left| \left\{ t \, \middle| \, author(t) = u_1 \wedge score(t) > 0 \right\} \right| > \left| \left\{ t \, \middle| \, author(t) = u_2 \wedge score(t) > 0 \right\} \right|$$

In [28]:
# Require tweetList constructed from previous section
import operator
from itertools import islice

# map user id to number of positive tweets
userToPositives = {}
for tweet in tweetList:
    score = tweet[2]
    if score > 0:
        userId = tweet[1]
        # in some cases user has multiple attributes as id
        if isinstance(userId, dict):
            userId = userId.values()[0]
            
        if not userToPositives.has_key(userId):
            userToPositives[userId] = 0
        userToPositives[userId] += 1

userToPositives = sorted(userToPositives.iteritems(), key=operator.itemgetter(1), reverse=True)
list(islice(userToPositives, 3))

[(1243188937, 28), (1601537689, 25), (u'2229663685', 22)]

**(1 point) Assignment question #3.3:** 
We have discussed earlier that our system is fragile to negations: it will score the expression *not beautiful* as positive because it only detects the word beautiful as positive. 
More generally, we consider that the token **not** changes the polarity of a given token: **not beautiful** becomes negative, while **not bad** becomes positive.  
Modify the function `get_sentiment_score(text)` to detect the changes of polarity due to the token **not**.

In [ ]:
# with negation, not considering punctuation
def get_sentiment_score(text):
    # tokenize and lemmatize the current tweet
    tokens = nltk.word_tokenize(text)
    tweet = [lmtzr.lemmatize(x.lower()) for x in tokens if len(x) >= 3]

    # calculate the sentiment score
    score = 0
    # the nagation value is flipped every time a "not" is encountered
    neg = 1
    for word in tweet:
        if word == "not":
            neg = -neg
        else:
            if word in positive_words:
                score = score + neg
            if word in negative_words:
                score = score - neg
    return score

Bonus assignments questions
---
Bonus assignment questions earn you extra marks if you solve this assignment correctly, no penalty is inflicted if you do not solve it. 
Note that the total grade of SMA assignments cannot exceed 10 points, therefore the bonus point can only be used to compensate for another question which you did not solve correctly.

**(0.25 additional points) Bonus question #1.** 
Based on the scores calculated in **Assignment #3.1**, plot the temporal evolution of the counts of positive and negative tweets. The date a tweet was emitted is found in the field *created_at*. 
Divide the temporal extent of your dataset into 100 timeslices. 
The temporal extent of the dataset is from the creation date of the first tweet to the creation date of the last tweet. 
Count how many positive and how many negative tweets you have in each timeslice. 
Plot these counts on a graphic resembling this one:  
![caption](files/desired-plot.png)  
**HINT:** the graphic above is not based on real data. Your actual curves might **NOT** look like this one. Its purpose is just to show you the expected form of the graphic.

In [25]:
# code or answer for question. In case of plain text answer, change the type of cell to "Raw Text" using the toolbar.

/students/u5319850/miniconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


**(0.25 additional points) Bonus question #2: Polarized communities** We want to know if there are polarized communities in our social graph. Are there closely linked groups of users who have similar sentiment polarities? We calculate the sentiment polarity of a user as the numbre of positive tweets emitted, from which we substract the number of negative tweets. Formally:
$$sentiment\_polarity(u) = \left| \left\{ t \, \middle| \, author(t) = u \wedge score(t) > 0 \right\} \right| - \left| \left\{ t \, \middle| \, author(t) = u \wedge score(t) < 0 \right\} \right| \enspace.$$  
To visually detect the polarized communities, we want to plot the same graph as in [tutorial 1](http://nbviewer.ipython.org/url/rizoiu.eu/sna-lab-ipython/tutorial-1-construct-social-graph.ipynb) at *Step 3*, but with the colors of nodes representing their polarity: `blue` for a negative polarity and `red` for a positive polarity. 
Are nodes colored similarly clustered close together?

In [16]:
# code or answer for question. In case of plain text answer, change the type of cell to "Raw Text" using the toolbar.